In [6]:
import pandas as pd
import openpyxl
import datetime
import os

In [7]:
path3 = r'D:\Python\Financial\Data\微信支付账单(20220921-20221209).csv'

path_account = r'D:\@生活\台帐\个人财务管理.xlsx'
path_merge = r'本次写入数据.csv'


In [8]:
def strip_in_data(data):  # 把列名中和数据中首尾的空格都去掉。
    data = data.rename(columns={column_name: column_name.strip() for column_name in data.columns})
    data = data.applymap(lambda x: x.strip().strip('¥') if isinstance(x, str) else x)
    return data

In [9]:
def read_data_wx1(path):  # 获取微信数据
    wx1 = pd.read_csv(path,header=16, skipfooter=0, encoding='utf_8',encoding_errors='ignore')
    wx1.iloc[:, 5] = wx1.iloc[:, 5].str[1:].astype('float64')  # 数据类型更改
    for i in wx1.index:       #将收入赋负号
        if wx1.iloc[i,4] == '收入':
            wx1.iloc[i,5]= - wx1.iloc[i,5]
    wx1 = wx1[ ~ wx1['支付方式'].str.contains('银行') ]
    wx1 = wx1.iloc[:, [0, 0, 5, 1, 2, 3]]  # 按顺序提取所需列
    wx1 = strip_in_data(wx1)  # 去除列名与数值中的空格。
    wx1.columns = ['交易日期','交易时间','支出','摘要','对方户名','交易详情']
    wx1.insert(3, '收入', "")  
    wx1.insert(7, '数据来源', "微信")# 添加微信来源标识
    wx1.iloc[:, 0] = wx1.iloc[:, 0].astype('str')  # 数据类型更改
    wx1.iloc[:, 0] = wx1.iloc[:, 0].astype('datetime64')  # 数据类型更改
    wx1.iloc[:, 1] = wx1.iloc[:, 1].astype('datetime64')  # 数据类型更改
    wx1.iloc[:, 2] = wx1.iloc[:, 2].astype('float64')  # 数据类型更改
    wx1.reset_index(drop=True, inplace=True)

    for i in wx1.index:       #将“支出/收入”单列转换为支出、收入分别一列
        if wx1.iloc[i,2] < 0:
            wx1.iloc[i,3]= -wx1.iloc[i,2]
            wx1.iloc[i,2]= 0
        else:
            wx1.iloc[i,3]= 0 
    wx1.iloc[:, 3] = wx1.iloc[:, 3].astype('float64')  # 数据类型更改
    #print(wx1)
    #print(wx1.dtypes)
    len2 = len(wx1)
    print("成功读取 " + str(len2) + " 条「微信」账单数据\n") 
    return(wx1)


In [10]:
data_wx1 = read_data_wx1(path3)


data_merge = data_wx1
merge_list = data_merge.values.tolist()

data_merge.to_csv(path_merge)

workbook = openpyxl.load_workbook(path_account)  # openpyxl读取账本文件
sheet = workbook['财务记录']
maxrow = sheet.max_row  # 获取最大行
print('\n「明细」 sheet 页已有 ' + str(maxrow) + ' 行数据，将在末尾写入数据')
for row in merge_list:
    sheet.append(row)  # openpyxl写文件

workbook.save(path_account)  # 保存
print("\n成功将数据写入到 " + path_account)

成功读取 51 条「微信」账单数据


「明细」 sheet 页已有 3194 行数据，将在末尾写入数据

成功将数据写入到 D:\@生活\台帐\个人财务管理.xlsx
